# Example notebook for SPICE

The API documentation can be found at /docs/api for a description of each endpoint and the fields: 
    https://spice.epfl.ch/api/docs

In [ ]:
from spice_client import ApiClient, Configuration
from spice_client.helpers.wrapper import get_jwt_token
from spice_client.api.default_api import DefaultApi
import pandas as pd
import geopandas as gpd
import json

SERVER = "https://spice.epfl.ch"

auth_token = get_jwt_token(SERVER)
config = Configuration(host=SERVER, access_token=auth_token)
api = DefaultApi(ApiClient(configuration=config))

# Test getting projects, to see if we're connected
print("Connected") if api.get_all_projects() is not None else print("Not connected")

In [ ]:
# Get all projects
print(api.get_all_projects())

# We can also do a filter (but we have to serialise the filter terms into JSON first):

print("\nWith filter:", api.get_all_projects(filter=json.dumps({'name': "Ag"})))

In [ ]:
# Use the ID to get one response (usually more data is returned)

ag_frost_proection = api.get_one_project('00aa40f4-4122-472d-890d-ba40671b819b')
ag_frost_proection

In [ ]:
api.get_one_location('7ae01251-81ce-4f44-8e31-63a92ac76ff6')

In [ ]:
for experiment in sorted(api.get_all_experiments(), key=lambda x: x.name):
    print(f"{experiment.name:35} {experiment.id}")

In [ ]:
# With this result, make a pandas dataframe from the results if you want
pd.DataFrame.from_dict([x.to_dict() for x in api.get_all_experiments()])

In [ ]:
exp139 = api.get_one_experiment("4642971c-6717-4b3b-b8f2-462b3a557302")  # A UUID from the ID column above

In [ ]:
# Experiments, if they have results, are organised by experiment.results.trays[].wells[]. For example:

exp139.results.trays[0].wells[0].to_dict()  # This is likely well A1 from the first tray (0) -- As a JSON/Dictionary

In [ ]:
# We could search for a specific well if desired, from tray 0
TRAY = 1
WELL = 'H2'

for well in exp139.results.trays[TRAY].wells:
    if well.coordinate == WELL:
        result = well

print(result)

In [ ]:
# We could then get more information about the sample, we have the ID field in the result.sample.id
print(result.sample.id)

# Get the sample from the sample endpoint. Here we have all the treatments, and all
# of the results from any experiments done to each sample/treatment.
# print(api.get_one_sample(result.sample.id))

# Querying by sample gives us some aggregated statistics, for example, grouping by dilution on each treatment
for treatment in api.get_one_sample(result.sample.id).treatments:
    print(f"Treatment: {treatment.name}")
    print(f"Dilution summary: {treatment.dilution_summaries}")
    print()

In [ ]:
# We can also search by sample but it might be easier to navigate to find the ID from within the UI.
# By default, our limit is 10, but we can alter the range (and filter, and paginate too if you would like), here we request 1000
for sample in api.get_all_samples(range=json.dumps([0,20])):
    print(f"{sample.name:65} {sample.id}")

print(api.get_one_sample("0092be07-d661-4a04-a1ce-e7b428c8da4c"))